In [13]:
!pip install sqlalchemy pymysql -q

In [14]:
!pip install sqlalchemy sql_magic -q

In [17]:
%load_ext sql_magic

The sql_magic extension is already loaded. To reload it, use:
  %reload_ext sql_magic


In [47]:
%sql

UsageError: Line magic function `%sql` not found.


In [1]:
import pandas as pd
import numpy as np


## Write pandas df to database

In [2]:
user="cornell"
password="Pitney@1a"
host="gec-dwh.ctz5iw1v2fh7.us-west-2.rds.amazonaws.com"
db="cornell"

conn = f"mysql+pymysql://{user}:{password}@{host}/{db}"

In [3]:
from sqlalchemy import create_engine

engine = create_engine(f"mysql+pymysql://{user}:{password}@{host}/{db}")

In [4]:
from pandas.io import sql


## Query database

In [6]:
result = engine.execute('show databases')
for _r in result:
    print(_r)

('information_schema',)
('cornell',)


In [147]:
result = engine.execute('show tables in cornell')
for _r in result:
    print(_r)

('deliver_package_info',)
('input_df',)
('ne_delivery_data_by_day_2018',)
('ne_returns_data_by_day_2018',)
('return_package_info_update',)
('test_df',)


## Query first 10 delivery package data & fields name

In [61]:
result = engine.execute('select * from cornell.ne_delivery_data_by_day_2018 limit 10')
for _r in result:
    print(_r)

(datetime.date(2018, 9, 30), 'Delivery', '362970510', '0.1950', 'Client - 2462', '11096', 'IND - PartnerFacility', '46037')
(datetime.date(2018, 9, 6), 'Delivery', '358082836', '0.2300', 'Client - 1216', '11096', 'Courier - Plainville', '2762')
(datetime.date(2018, 8, 14), 'Delivery', '355433187', '1.6600', 'Client - 1216', '11096', 'Courier - Secaucus', '7094')
(datetime.date(2018, 7, 11), 'Delivery', '350299758', '0.8800', 'Client - 1216', '11096', 'IND - PartnerFacility', '46037')
(datetime.date(2018, 9, 6), 'Delivery', '358082834', '0.4850', 'Client - 1216', '11096', 'IND - PartnerFacility', '46037')
(datetime.date(2018, 9, 6), 'Delivery', '358082830', '15.1200', 'Client - 1216', '11096', 'IND - PartnerFacility', '46037')
(datetime.date(2018, 7, 9), 'Delivery', '350140996', '1.3400', 'Client - 1216', '11096', 'IND - PartnerFacility', '46037')
(datetime.date(2018, 7, 11), 'Delivery', '350286964', '0.1850', 'Client - 1216', '11096', 'IND - PartnerFacility', '46037')
(datetime.date(20

In [35]:
result = engine.execute('describe cornell.deliver_package_info')
for _r in result:
    print(_r)

('index', 'bigint(20)', 'YES', 'MUL', None, '')
('origindate', 'date', 'YES', '', None, '')
('originfacilityzip', 'text', 'YES', '', None, '')
('destinationfacilityzip', 'text', 'YES', '', None, '')
('count', 'bigint(20)', 'YES', '', None, '')
('origin_major_city', 'text', 'YES', '', None, '')
('origin_county', 'text', 'YES', '', None, '')
('origin_state', 'text', 'YES', '', None, '')
('origin_lat', 'double', 'YES', '', None, '')
('origin_lng', 'double', 'YES', '', None, '')
('dest_major_city', 'text', 'YES', '', None, '')
('dest_county', 'text', 'YES', '', None, '')
('dest_state', 'text', 'YES', '', None, '')
('dest_lat', 'double', 'YES', '', None, '')
('dest_lng', 'double', 'YES', '', None, '')


## How many delivery data do we have

In [10]:
result = engine.execute('select count(*) from cornell.ne_delivery_data_by_day_2018')
for _r in result:
    print(_r)

(61916290,)


## Daily demand of delivery packages

In [ ]:
result = engine.execute('select origindate, count(*) from cornell.ne_delivery_data_by_day_2018 group by origindate')
for _r in result:
    print(_r)

### The number of deliver packages from customer warehouse to USPS & store in a new table "deliver_package_info"

In [49]:
# The # of packages from customer warehouse to USPS 
deliver_result = engine.execute('select origindate,originfacilityzip,destinationfacilityzip,count(*) from cornell.ne_delivery_data_by_day_2018 \
                        group by originfacilityzip,destinationfacilityzip \
                        order by origindate')
deliver_df = pd.DataFrame(deliver_result.fetchall(),columns=['origindate','originfacilityzip','destinationfacilityzip','count'])

In [50]:
deliver_df

,origindate,originfacilityzip,destinationfacilityzip,count
0,2018-01-01,10974,07097,14
1,2018-01-01,21057,24012,8
2,2018-01-01,21057,06492,4
3,2018-01-01,21057,12288,2
4,2018-01-03,21057,3820,4
5,2018-01-03,08837,,2
6,2018-01-03,06492,,4
7,2018-01-03,10533,13760,6
8,2018-01-03,11096,,8133
9,2018-01-04,06492,12205,1379


In [14]:
deliver_df.to_sql(con=engine, name='deliver_package_info', 
                if_exists='replace')

## Query first 10 return package data & fields name

In [ ]:
result = engine.execute('select * from cornell.ne_returns_data_by_day_2018 limit 10')
for _r in result:
    print(_r)

In [ ]:
result = engine.execute('describe cornell.ne_returns_data_by_day_2018')
for _r in result:
    print(_r)

## How many return data do we have

In [9]:
result = engine.execute('select count(*) from cornell.ne_returns_data_by_day_2018')
for _r in result:
    print(_r)

(33276614,)


## Daily demand of return packages

In [ ]:
result = engine.execute('select origindate, count(*) from cornell.ne_returns_data_by_day_2018 group by origindate')
for _r in result:
    print(_r)

### The number of daily return packages from USPS to customer warehouse & store in a new table "return_package_info"

In [15]:
return_result = engine.execute('select origindate,originfacilityzip,destinationfacilityzip,count(*) from cornell.ne_returns_data_by_day_2018 \
                        group by originfacilityzip,destinationfacilityzip \
                        order by origindate')
return_df = pd.DataFrame(return_result.fetchall(),columns=['origindate','originfacilityzip','destinationfacilityzip','count'])




In [17]:
return_df.to_sql(con=engine, name='return_package_info', if_exists='replace')

## geodecode zip3

#### zip3 reference dict

In [6]:
zip3_dic={}
zip3_file='/home/fusion/jupyterhub-notebooks/Cornell_PB_NE_Hub/references/zip3_centroids.csv'
import csv
with open(zip3_file,'r',encoding="utf-8") as f:
    reader = csv.reader(f)
    for row in reader:
        zip3_dic[row[2]]=[row[3],row[4],row[5]]


### decodezip3 function

In [51]:
def decodezip3(zip3,reference):
    res=[]
    for i in zip3:
        if len(i)==3:
            i='00'+i
        elif len(i)==4:
            i='0'+i
            
        zipp=i[0:3]
        if not i:
            tmp=[zipp,'','','']
        else:
            if zipp in reference.keys():
                zipinfo=reference[zipp]
                tmp=[zipp,zipinfo[0],zipinfo[1],zipinfo[2]]
            else:
                tmp=[zipp,'','','']    
        res.append(tmp)
    return res


### decode deliver - origin

In [36]:
result=engine.execute('select origindate,originfacilityzip,destinationfacilityzip,count from deliver_package_info')

In [37]:
deliver_df = pd.DataFrame(result.fetchall(),columns=['origindate','originfacilityzip','destinationfacilityzip','count'])

In [38]:
deliver_tmp=decodezip3(deliver_df['originfacilityzip'],zip3_dic)

\N \N
\N \N
\N \N
\N \N


In [39]:
tmp_df=pd.DataFrame(deliver_tmp,columns=['origin_zip3','origin_state','origin_lat','origin_lng'])

In [40]:
deliver_df=deliver_df.join(tmp_df)

In [41]:
deliver_df

,origindate,originfacilityzip,destinationfacilityzip,count,origin_zip3,origin_state,origin_lat,origin_lng
0,2018-01-01,10974,07097,14,109,NY,41.1434616274125,-74.1238460876456
1,2018-01-01,21057,24012,8,210,MD,39.4748982668505,-76.2084143365847
2,2018-01-01,21057,06492,4,210,MD,39.4748982668505,-76.2084143365847
3,2018-01-01,21057,12288,2,210,MD,39.4748982668505,-76.2084143365847
4,2018-01-03,21057,3820,4,210,MD,39.4748982668505,-76.2084143365847
5,2018-01-03,08837,,2,088,NJ,40.6240218011197,-74.8843774239638
6,2018-01-03,06492,,4,064,CT,41.3447619004598,-73.068704647852
7,2018-01-03,10533,13760,6,105,NY,41.2922361699564,-73.7604420299132
8,2018-01-03,11096,,8133,110,NY,40.7220752067917,-73.7068341419121
9,2018-01-04,06492,12205,1379,064,CT,41.3447619004598,-73.068704647852


### decode deliver - dest

In [42]:
destination_tmp=decodezip3(deliver_df['destinationfacilityzip'],zip3_dic)

In [43]:
dest_df=pd.DataFrame(destination_tmp,columns=['dest_zip3','dest_state','dest_lat','dest_lng'])


In [44]:
deliver_df=deliver_df.join(dest_df)

In [45]:
deliver_df

,origindate,originfacilityzip,destinationfacilityzip,count,origin_zip3,origin_state,origin_lat,origin_lng,dest_zip3,dest_state,dest_lat,dest_lng
0,2018-01-01,10974,07097,14,109,NY,41.1434616274125,-74.1238460876456,070,NJ,40.5833124911422,-74.2703150322948
1,2018-01-01,21057,24012,8,210,MD,39.4748982668505,-76.2084143365847,240,VA,37.2706334855373,-79.9411734251418
2,2018-01-01,21057,06492,4,210,MD,39.4748982668505,-76.2084143365847,064,CT,41.3447619004598,-73.068704647852
3,2018-01-01,21057,12288,2,210,MD,39.4748982668505,-76.2084143365847,122,NY,42.6472800295118,-73.7516930834563
4,2018-01-03,21057,3820,4,210,MD,39.4748982668505,-76.2084143365847,038,NH,43.0727535404586,-70.8040595470843
5,2018-01-03,08837,,2,088,NJ,40.6240218011197,-74.8843774239638,,,,
6,2018-01-03,06492,,4,064,CT,41.3447619004598,-73.068704647852,,,,
7,2018-01-03,10533,13760,6,105,NY,41.2922361699564,-73.7604420299132,137,NY,42.2343589257587,-75.5390075382465
8,2018-01-03,11096,,8133,110,NY,40.7220752067917,-73.7068341419121,,,,
9,2018-01-04,06492,12205,1379,064,CT,41.3447619004598,-73.068704647852,122,NY,42.6472800295118,-73.7516930834563


In [46]:
deliver_df.to_sql(con=engine, name='deliver_package_info', if_exists='replace')

### decode return origin

In [47]:
result=engine.execute('select origindate,originfacilityzip,destinationfacilityzip,count from return_package_info_update')
return_df = pd.DataFrame(result.fetchall(),columns=['origindate','originfacilityzip','destinationfacilityzip','count'])



In [48]:
return_tmp=decodezip3(return_df['originfacilityzip'],zip3_dic)

\N \N
\N \N
0 0
\N \N
\N \N
\N \N
0 0
0 0
\N \N
\N \N
0 0
\N \N
\N \N
0 0
\N \N
0 0
\N \N
\N \N
0 0
0 0
\N \N
\N \N
\N \N
0 0
0 0
\N \N
0 0
\N \N
0 0
\N \N
\N \N
\N \N
\N \N
0 0
\N \N
\N \N
\N \N
\N \N
0 0
\N \N
\N \N
\N \N
0 0
\N \N
\N \N
\N \N
\N \N
\N \N
0 0
\N \N
0 0
\N \N
0 0
\N \N
\N \N
\N \N
0 0
0 0
0 0
0 0
0 0
\N \N
\N \N
0 0
\N \N


In [28]:
tmp_df=pd.DataFrame(return_tmp,columns=['origin_zip3','origin_state','origin_lat','origin_lng'])

In [29]:
return_df=return_df.join(tmp_df)

### decode return destination

In [30]:
return_tmp=decodezip3(return_df['destinationfacilityzip'],zip3_dic)

In [31]:
tmp_df=pd.DataFrame(return_tmp,columns=['dest_zip3','dest_state','dest_lat','dest_lng'])

In [32]:
return_df=return_df.join(tmp_df)

In [33]:
return_df

,origindate,originfacilityzip,destinationfacilityzip,count,origin_zip3,origin_state,origin_lat,origin_lng,dest_zip3,dest_state,dest_lat,dest_lng
0,2018-01-02,15930,27801,2,159,PA,40.3278437185358,-78.9138177538114,278,NC,35.911301017443,-77.7362497363615
1,2018-01-02,23222,07836,1,232,VA,37.5392326292499,-77.4374487131616,078,NJ,40.9151229858398,-74.5807952880859
2,2018-01-02,4092,07836,4,040,ME,43.530478933588,-70.9212281508382,078,NJ,40.9151229858398,-74.5807952880859
3,2018-01-02,01472,52404,1,014,MA,42.5853946189292,-71.8170603313981,524,IA,41.9754515759122,-91.6587232808614
4,2018-01-02,15930,17699,2,159,PA,40.3278437185358,-78.9138177538114,176,PA,40.0730558106315,-76.3141886839793
5,2018-01-02,20676,21704,1,206,MD,38.6176487130023,-76.8552682490031,217,MD,39.4477721717067,-77.3342903346635
6,2018-01-02,08880,16368,1,088,NJ,40.6240218011197,-74.8843774239638,163,PA,41.4542864255809,-79.6519755218344
7,2018-01-02,20676,46037,1,206,MD,38.6176487130023,-76.8552682490031,460,IN,40.2490326631515,-85.6604766032736
8,2018-01-02,07721,21704,1,077,NJ,40.3606127258978,-74.0780750505036,217,MD,39.4477721717067,-77.3342903346635
9,2018-01-02,15930,30349,1,159,PA,40.3278437185358,-78.9138177538114,303,GA,33.7554224110405,-84.3904423185221


In [34]:
# return_df.to_sql(con=engine, name='return_package_info', if_exists='replace',dtype=['datetime.date','text','text','bigint(20)','text','text','text','double','double','text','text','text','double','double'])
return_df.to_sql(con=engine, name='return_package_info', if_exists='replace')


## Detect zip code which cannot be decode

In [5]:
zip3_dic={}
zip3_file='/home/fusion/jupyterhub-notebooks/Cornell_PB_NE_Hub/references/zip3_centroids.csv'
import csv
with open(zip3_file,'r',encoding="utf-8") as f:
    reader = csv.reader(f)
    for row in reader:
        zip3_dic[row[2]]=[row[3],row[4],row[5]]

In [6]:
def decodezip3(zip3,reference):
    res=[]
    for i in zip3:
        if len(i)==3:
            i='00'+i
        elif len(i)==4:
            i='0'+i
            
        zipp=i[0:3]
        if not i:
            tmp=[zipp,'','','']
        else:
            if zipp in reference.keys():
                zipinfo=reference[zipp]
                tmp=[zipp,zipinfo[0],zipinfo[1],zipinfo[2]]
            else:
                print(zipp)
                tmp=[zipp,'','','']    
        res.append(tmp)
    return res


In [13]:
r709=engine.execute('select originfacilityzip, destinationfacilityzip, count(*) from ne_returns_data_by_day_2018 where originfacilityzip REGEXP \'^709\' or destinationfacilityzip REGEXP \'^709\'')
for _r in r709:
    print(_r)

('7094', '92407', 1711093)
